In [1]:
import pickle
import numpy as np
import json
import os
from collections import defaultdict
import collections
import pandas as pd

In [2]:
wikihow_part_dir = '/mount/arbeitsdaten/jp-silberer/ernie_vil/data/pkg_data/wikiHow/partial'
wikihow_full_dir = '/mount/arbeitsdaten/jp-silberer/ernie_vil/data/pkg_data/wikiHow/full'

howto_dir = '/mount/arbeitsdaten/jp-silberer/ernie_vil/data/pkg_data/howto100m'
cae_dir = '/mount/arbeitsdaten/jp-silberer/ernie_vil/data/pkg_data/cae'

In [3]:
howto_df = pd.read_csv(os.path.join(howto_dir, 'HowTo100M_v1.csv'))
task_ids_df = pd.read_csv(os.path.join(howto_dir, 'task_ids.csv'), delimiter='\t', names=['task_id', 'task_title'])

In [4]:
cae_by_vid = json.load(open(os.path.join(cae_dir,'single_result_verbs_video_clips_by_vid.json'), 'r'))
cae_vids = list(cae_by_vid.keys())

In [5]:
# get task_id to task_title information
taskid2title_df = pd.merge(howto_df, task_ids_df, on='task_id')

# get all cae
cae_taskid2title_df = taskid2title_df[taskid2title_df['video_id'].isin(cae_vids)]

In [6]:
taskid2title_df.head()

,video_id,category_1,category_2,rank,task_id,task_title
0,nVbIUDjzWY4,Cars & Other Vehicles,Motorcycles,27,52907,Paint a Motorcycle
1,rwmt7Cbuvfs,Cars & Other Vehicles,Motorcycles,99,52907,Paint a Motorcycle
2,HnTLh99gcxY,Cars & Other Vehicles,Motorcycles,35,52907,Paint a Motorcycle
3,RAidUDTPZ-k,Cars & Other Vehicles,Motorcycles,10,52907,Paint a Motorcycle
4,tYQoPHwNkho,Cars & Other Vehicles,Motorcycles,18,52907,Paint a Motorcycle


In [7]:
def get_domain2tasktitles(cae_taskid2title_df):
    categories = set(cae_taskid2title_df['category_1'].to_list())
    result = {}

    for cat in categories:
        result[cat] = list(set(cae_taskid2title_df[cae_taskid2title_df['category_1']==cat]['task_title'].to_list()))
    return result

def get_tasktitles2domain(domain2tasktitlles):
    tasktitles2domain = {}
    
    for d, tasktitles in domain2tasktitlles.items():
        for t in tasktitles:
            t = 'How to ' + t
            tasktitles2domain[t] = d
    return tasktitles2domain


domain2tasktitlles = get_domain2tasktitles(cae_taskid2title_df)
tasktitles2domain = get_tasktitles2domain(domain2tasktitlles)

# WikiHow files

## Full WikiHow version (num_tasks: 215, 365)

### (Already processed: article_id_to_title.txt, step_label_text.json) 

In [3]:
wikihow_all_df = pd.read_csv(os.path.join(wikihow_full_dir, 'wikihowAll.csv'))

In [4]:
wikihow_full_df = wikihow_all_df.dropna()
wikihow_full_tasknames = wikihow_full_df['title'].to_list()
print(len(wikihow_full_tasknames))
wikihow_full_df.head()

214294


,headline,title,text
0,"\nKeep related supplies in the same area.,\nMa...",How to Be an Organized Artist1,"If you're a photographer, keep all the necess..."
1,\nCreate a sketch in the NeoPopRealist manner ...,How to Create a Neopoprealist Art Work,See the image for how this drawing develops s...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",How to Be a Visual Effects Artist1,It is possible to become a VFX artist without...
3,\nStart with some experience or interest in ar...,How to Become an Art Investor,The best art investors do their research on t...
4,"\nKeep your reference materials, sketches, art...",How to Be an Organized Artist2,"As you start planning for a project or work, ..."


In [7]:
wikihow_full_headlines = wikihow_full_df['headline'].to_list()
headlines = wikihow_full_headlines[0].split('\n')

filtered_task_steps = [step.rstrip(',') for step in headlines if len(step) > 0]
print(filtered_task_steps)

['Keep related supplies in the same area.', 'Make an effort to clean a dedicated workspace after every session.', 'Place loose supplies in large, clearly visible containers.', 'Use clotheslines and clips to hang sketches, photos, and reference material.', 'Use every inch of the room for storage, especially vertical space.', 'Use chalkboard paint to make space for drafting ideas right on the walls.', 'Purchase a label maker to make your organization strategy semi-permanent.', 'Make a habit of throwing out old, excess, or useless stuff each month.']


In [6]:
# 1. produce step_label_text.json and article_id_to_title.txt (\t)
wikihow_full_article_id_to_title = []
step_labels = []

for index, row in wikihow_full_df.iterrows():
    task_steps = row['headline'].split('\n')
    filtered_task_steps = [step.rstrip(',') for step in task_steps if len(step) > 0]
    step_labels.append(task_steps)
    article_id_title = str(index) + '\t' + row['title']
    wikihow_full_article_id_to_title.append(article_id_title)

# write step labels
path = os.path.join(wikihow_full_dir, 'step_label.json')
with open(path, 'w') as f:
    json.dump(step_labels, f)

path = os.path.join(wikihow_full_dir, 'article_id_to_title.txt')
with open(path, 'w') as f:
    for a in wikihow_full_article_id_to_title:
        f.write(a+'\n')

## Partial WikiHow version (num_tasks: 1053, used by Paprika)

In [4]:
# insepct already extracted step features
with open(os.path.join(wikihow_part_dir, 's3d_text_feat', 'step_embeddings.pickle'), 'rb') as f:
    wikihow_step_feats = pickle.load(f)
    
with open(os.path.join(wikihow_part_dir, 's3d_text_feat', 'step_preprocessed.pickle'), 'rb') as f:
    wikihow_steps = pickle.load(f)

In [20]:
wikihow_step_feats.shape

(10588, 512)

In [15]:
for i in range(10):
    print(wikihow[0][i]['headline'])
    print(wikihow_steps[i])


Determine the correct proportions.
determine correct proportions

Understand that it will need to be strong, heavy and durable.
understand strong heavy durable

Make a level surface.
make level surface

Now lay your first sheet of mid grade plywood or MDF on top of the 2x4's.
lay sheet mid grade plywood mdf 2x4's

Start laying the inside grid of squares.
start laying inside grid squares

Build your outside box frame, use the 1x2 lumber, pre-drill holes or use a brad nailer and glue.
build outside box frame use 1x2 lumber pre drill holes use brad nailer glue

Cut your 1x2 lumber to the proper lengths for the grid, use long pieces the length of the bench and short 1x2's across the width.


cut 1x2 lumber proper lengths grid use long pieces length bench short 1x2's width

Place the short strips at their pre-determined location of the grid against the outside frame along the length of the bench.
place short strips pre determined location grid outside frame length bench

With the short str

In [13]:
# the version Paprika use
with open(os.path.join(wikihow_part_dir, 'step_label_text.json'), 'r') as f:
    wikihow_steps = json.load(f)

with open(os.path.join(wikihow_part_dir, 'article_id_to_title.txt'), 'r') as f:
    article_id_to_wikihow_taskname = {
        int(line.rstrip().split('\t')[0]): line.rstrip().split('\t')[1] for line in f.readlines()}

with open(os.path.join(wikihow_part_dir, 'article_id_to_title.txt'), 'r') as f:
    wikihow_taskname_to_article_id = {
        line.rstrip().split('\t')[1]: int(line.rstrip().split('\t')[0]) for line in f.readlines()}

In [13]:
wikihow_tasknames = list(wikihow_taskname_to_article_id.keys())
print(len(wikihow_tasknames))

1053


In [108]:
a_id = wikhow_taskname_to_article_id.get('How to Plant Parsnips')
wikihow_steps[a_id]

[{'headline': '\nPrepare the site.',
  'text': ' Loosen the soil with a fork and remove any weeds or large stones. Use a rake to create a level surface. Use the head of the rake to lightly press down the soil and lightly rake the surface once more.;\n',
  'sep_id': 1433133},
 {'headline': '\nLay a piece of string on the soil as a guide to make sure you dig the row in a straight line.',
  'text': ' You will also want to lay a plank of wood next to the line that you just made. Line the plank of wood up right next to the line that was formed by laying the string out as a guide.\n\n',
  'sep_id': 1433137},
 {'headline': '\nUse a cane, or other tool, to dig a narrow hole that you will place the seeds in.',
  'text': ' The narrow hole that you are digging should ideally be about 1 centimeter (0.4\xa0in) deep in length.\n\n',
  'sep_id': 1433138},
 {'headline': '\nUsing a hoe, gently cover the seeds with soil.',
  'text': ' Then, using the head of the metal rake, lightly firm the soil.\n\n',


In [14]:
def get_taskid2stepid(wikihow):
    step_id = 0
    article_po_to_step_id = defaultdict()
    step_id_to_headline = defaultdict()
    article_to_step_id = defaultdict(list)

    for article_id in range(len(wikihow)):
        for article_step_idx in range(len(wikihow[article_id])):
            article_po_to_step_id[(article_id, article_step_idx)] = step_id
            step_id_to_headline[step_id] = wikihow[article_id][article_step_idx]['headline'].strip("\n")
            article_to_step_id[article_id].append(step_id)
            step_id += 1
    total_num_steps = len(article_po_to_step_id)
    return article_po_to_step_id, step_id_to_headline

In [15]:
article_po_to_step_id, step_id_to_headline, article_po_to_step_id = get_taskid2stepid(wikihow)

In [16]:
# node2step & step2node (defaultdict)
with open(os.path.join(wikihow_part_dir, 'node2step.pickle'), 'rb') as f:
    node2step = pickle.load(f)

with open(os.path.join(wikihow_part_dir, 'step2node.pickle'), 'rb') as f:
    step2node = pickle.load(f)

In [17]:
num_steps = len(step2node) # 10,588
# TODO: quality check if the clustering of steps makes sense
num_nodes = len(node2step) # 10,388

# Obtain the video subset for analysis
- select 5 tasks across 13 domains, in total 65 task titles, resulting in 831 videos
- ?% of CAE tasks has exact overlap with
    - wikihow FULL 67.49
    - wikihow PARTIAL 2.82

In [19]:
def get_videoid2taskname(video_meta_df, task_ids_df):        
    task_id_to_task_name_original_map = dict()
    for index, row in task_ids_df.iterrows():
        task_id = row[0]
        task_name = row[1]
        task_id_to_task_name_original_map[task_id] = task_name
    
    video_id_to_task_name = dict()
    for index, row in video_meta_df.iterrows():
        video_id = row['video_id']
        task_id = row['task_id']
        video_id_to_task_name[video_id] = 'How to ' + task_id_to_task_name_original_map[task_id]
    
    return video_id_to_task_name

video_id_to_task_name = get_videoid2taskname(howto_df, task_ids_df)

In [18]:
saved_path = '/mount/projekte/jp-silberer/hyuyang/paprika/S3D_HowTo100M/sampled_vids.txt'
with open(saved_path, 'r') as f:
    sampled_vids = [line.rstrip() for line in f.readlines()]

In [46]:
# sampled videos across domains
domains = cae_taskid2title_df[cae_taskid2title_df['video_id'].isin(sampled_vids)]['category_1'].to_list()
collections.Counter(domains).most_common()

[('Food and Entertaining', 208),
 ('Home and Garden', 139),
 ('Hobbies and Crafts', 126),
 ('Cars & Other Vehicles', 56),
 ('Arts and Entertainment', 49),
 ('Pets and Animals', 45),
 ('Education and Communications', 41),
 ('Sports and Fitness', 40),
 ('Family Life', 36),
 ('Holidays and Traditions', 26),
 ('Personal Care and Style', 23),
 ('Health', 23),
 ('Computers and Electronics', 19)]

In [33]:
# sanity check on the overlap of wikihow task name and cae
cae_tasknames = set([video_id_to_task_name.get(vid) for vid in video_id_to_task_name.keys()])
print('CAE all tasks:', len(cae_tasknames))
subset_tasknames = set([video_id_to_task_name.get(vid) for vid in sampled_vids])

CAE all tasks: 25312


In [41]:
print('?% of CAE tasks has exact overlap with')
print('wikihow FULL:', len(cae_tasknames & set(wikihow_full_tasknames))/len(cae_tasknames)*100)
print('wikihow PARTIAL:', len(cae_tasknames & set(wikihow_tasknames))/len(cae_tasknames)*100)


?% of CAE tasks has exact overlap with
in wikihow FULL 67.48577749683943
in wikihow PARTIAL 2.8168457648546146


# PKG

## PKG helper functions

In [69]:
def find_node_transitions(adj_matrix, start_node):
    # DFS traversal
    num_nodes = adj_matrix.shape[0]  # Use the number of rows from the shape of the matrix
    transitions = []
    visited = [False] * num_nodes

    def dfs(node, transition):
        visited[node] = True
        transition.append(node)

        if not np.any(adj_matrix[node]):  # Use np.any to check if there are outgoing neighbors to serve as end nodes
            transitions.append(transition[:])  # Add a copy of the transition to the list

        for neighbor, connected in enumerate(adj_matrix[node]):
            if connected and not visited[neighbor]:
                dfs(neighbor, transition)

        visited[node] = False
        transition.pop()  # Remove the last node to backtrack and explore other options

    dfs(start_node, [])

    return transitions

def find_node_transitions_with_end_node(adj_matrix, start_node, end_node):
    # Q. Does it overcome the case where end node does not have any incoming edge?
    num_nodes = adj_matrix.shape[0]  # Use the number of rows from the shape of the matrix
    transitions = []
    visited = [False] * num_nodes

    def dfs(node, transition):
        visited[node] = True
        transition.append(node)

        if node == end_node or not np.any(adj_matrix[node]):  # Define end node or use np.any to check if there are outgoing neighbors
            transitions.append(transition[:])  # Add a copy of the transition to the list

        for neighbor, connected in enumerate(adj_matrix[node]):
            if connected and not visited[neighbor]:
                dfs(neighbor, transition)

        visited[node] = False # to 
        transition.pop()  # Remove the last node to backtrack and explore other options

    dfs(start_node, [])

    return transitions


def find_node_transitions_with_depth(adj_matrix, start_node, depth_count):
    # TODO: it would be better to take the score into account
    num_nodes = adj_matrix.shape[0]  # Use the number of rows from the shape of the matrix
    transitions = []
    visited = [False] * num_nodes

    def dfs(node, transition, depth):        
        visited[node] = True
        transition.append(node)

        if not np.any(adj_matrix[node]) or depth >= depth_count:  # Define end node or use np.any to check if there are outgoing neighbors
            transitions.append(transition[:])  # Add a copy of the transition to the list
            visited[node] = False  # Reset visited status to explore other options
            transition.pop()  # Remove the last node to backtrack
            return
        
        for neighbor, connected in enumerate(adj_matrix[node]):
            if connected and not visited[neighbor]:
                dfs(neighbor, transition, depth + 1)

        visited[node] = False  
        transition.pop()  # Remove the last node to backtrack and explore other options
        
    dfs(start_node, [], depth=0)

    return transitions

def find_node_transitions_within_range(adj_matrix, start_node, node_range=None):
    num_nodes = adj_matrix.shape[0]  # Use the number of rows from the shape of the matrix
    transitions = []
    visited = [False] * num_nodes

    def dfs(node, transition):        
        visited[node] = True
        transition.append(node)
            
        if not np.any(adj_matrix[node]):  # Define end node or use np.any to check if there are outgoing neighbors
            print('node meets end criteria', node)
            transitions.append(transition[:])  # Add a copy of the transition to the list
            visited[node] = False  # Reset visited status to explore other options
            transition.pop()  # Remove the last node to backtrack
            return
        
        for neighbor, connected in enumerate(adj_matrix[node]):
            # connected means > 0
            print('current node', node)
            if connected and not visited[neighbor] and neighbor in node_range:
                print('neighbor', neighbor)
                dfs(neighbor, transition)

        visited[node] = False  
        transition.pop()  # Remove the last node to backtrack and explore other options
        
    dfs(start_node, [])

    return transitions


# Quick test
# Example adjacency matrix as a NumPy array
adj_matrix = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0]
])

score_adj_matrix = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0.5, 1, 0.7],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0]
])

start_node = 3
end_node = 5
node_range = [1, 2, 3, 4]

transitions = find_node_transitions(adj_matrix, start_node)
print("Node Transitions with Start Node:", transitions)

# FIX ME, 6 should not appear
transitions = find_node_transitions_with_end_node(adj_matrix, start_node, end_node)
print("Node Transitions with Start and End Node:", transitions)

transitions = find_node_transitions_with_depth(adj_matrix, start_node, depth_count=2)
print("Node Transitions with Depth Count:", transitions)

# FIX ME, [3, 6] should not happen
transitions = find_node_transitions_within_range(score_adj_matrix, start_node, node_range=node_range)
print("Node Transitions within Node Range:", transitions)

Node Transitions with Start Node: [[3, 4], [3, 5, 6], [3, 6]]
Node Transitions with Start and End Node: [[3, 4], [3, 5], [3, 5, 6], [3, 6]]
Node Transitions with Depth Count: [[3, 4], [3, 5, 6], [3, 6]]
current node 3
current node 3
current node 3
current node 3
current node 3
neighbor 4
node meets end criteria 4
current node 3
current node 3
Node Transitions within Node Range: [[3, 4]]


In [63]:
# Get Step Sequences from Node Transitions
def get_step_sequence_by_node_sequence(node_sequences, node2step, step_id_to_headline):
    muliple_step_sequences = []
    for trans_seq in node_sequences:
#         print(trans_seq)
        mapped_step_sequence = []
        for n in trans_seq:
            steps = node2step[n]
            headlines = []
            for s in steps:
                headline = step_id_to_headline[s]
                headlines.append(headline)
            mapped_step_sequence.append(headlines)
        muliple_step_sequences.append(mapped_step_sequence)
    return muliple_step_sequences

# Local Graph

## Define task similarities

### Helper Function:

In [ ]:
from sklearn.cluster import AgglomerativeClustering

def get_tasktitles_clustering(task_feats, saved_dir, n_clusters, task_clustering_linkage, 
                              task_clustering_distance_thresh, task_clustering_affinity):
    clustering = AgglomerativeClustering(
        n_clusters=n_clusters, 
        linkage=task_clustering_linkage, 
        distance_threshold=task_clustering_distance_thresh, 
        affinity=task_clustering_affinity).fit(task_feats)
        # distance_threshold: 
        #   The linkage distance threshold above which, clusters will not be merged. 
    num_topics = clustering.n_clusters_

    topic2task, task2topic = defaultdict(), defaultdict()
    for cluster_id in range(num_topics):
        cluster_members = np.where(clustering.labels_ == cluster_id)[0]
        topic2task[cluster_id] = cluster_members
        for task_id in cluster_members:
            task2topic[task_id] = cluster_id
    
    with open(os.path.join(saved_dir, 'task_titles/topic2task.pickle'), 'wb') as f:
        pickle.dump(topic2task, f)
    with open(os.path.join(saved_dir, 'task_titles/task2topic.pickle'), 'wb') as f:
        pickle.dump(task2topic, f)  

    return topic2task, task2topic
    

with open(os.path.join(wikihow_part_dir, 'task_titles/s3d_text_feats/task_title_embeddings.pickle'), 'rb') as f:
    task_feats = pickle.load(f)

topic2task, task2topic = get_tasktitles_clustering(
                                                   task_feats,
                                                   saved_dir=wikihow_part_dir,
                                                   n_clusters=None,
                                                   task_clustering_linkage='average',
                                                   task_clustering_distance_thresh=0.5,
                                                   task_clustering_affinity='cosine')

In [8]:
with open(os.path.join(wikihow_part_dir, 'article_id_to_title.txt'), 'r') as f:
    article_idx_to_wikihow_taskname = {
        index: line.rstrip().split('\t')[1] for index, line in enumerate(f.readlines())}

In [40]:
print(len(topic2task))
c = 0
for tasks in topic2task.values():
    group_tasks = [article_idx_to_wikihow_taskname.get(t) for t in tasks]
    group_tasks_embeds = [task_feats[t] for t in tasks]
    c += 1
    print(group_tasks[:10])
    if c >= 5:
        break

864
['How to Build the Ultimate Tippmann Sniper Paintball Gun', 'How to Install a Flatline Barrel on a Tippmann A 5 Paintball Gun']
['How to Make Hair Spray1', 'How to Weave Hair', 'How to Wrap Hair']
['How to Make Fake Gold Coins', 'How to Make Fake Gold from Nickels']
['How to Care for Motorcycle Batteries', 'How to Change a Motorcycle Tire1', 'How to Make a Motorcycle Seat Pad1', 'How to Replace Motorcycle Hand Grips1', 'How to Synchronize Motorcycle Carburettors1']
['How to Crochet a Leaf1', 'How to Crochet a Ruffle Scarf1', 'How to Crochet a Star1', 'How to Crochet a Triple Picot Border1', 'How to Crochet an I Cord1', 'How to Crochet the Criss Cross Stitch1', 'How to Half Double Crochet (HDC)', 'How to Hold a Crochet Hook', 'How to Treble Crochet', 'How to Tunisian Crochet the Crossed Stitch']
['How to Create a No Dig Garden', 'How to Prepare Your Garden for Winter1']
['How to Organize Appointments on Your Android1', 'How to Organize Ribbons1', 'How to Organize Your Bills1']
['How

In [ ]:
# Check the dot product value of the group tasks

In [15]:
len(task_feats)

214294

### Within WikiHow:

### Between WikiHow & HowTo100M:

# Global Graph

## CAE_subtitles vs. HowTo100M_subtitles

In [64]:
# pkg (based o node id)
howto_pkg = np.load(os.path.join(howto_dir, 'subtitles/graph_output/PKG-criteria_topK-threshold_10-topK_3-agg_1000.npy'))
cae_pkg = np.load(os.path.join(cae_dir, 'subtitles/graph_output/PKG-criteria_topK-threshold_10-topK_3-agg_1000.npy'))

cae_pkg_matrix = np.asmatrix(cae_pkg)
howto_pkg_matrix = np.asmatrix(howto_pkg)

In [32]:
assert len(cae_pkg_matrix)==len(howto_pkg_matrix)

In [46]:
cae_pkg_matrix

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [47]:
len(np.argwhere(cae_pkg_matrix>0))

9537

In [48]:
len(np.argwhere(howto_pkg_matrix>0))

536807

### Decode Node Transitions from PKG 

In [77]:
# step 1: either find the start node of each task based on WikiHow or based on one video
def get_start_end_node_of_task(task_name):
    a_id = wikihow_taskname_to_article_id[task_name]
    step_seq = article_to_step_id[a_id]
    start_step_id = step_seq[0]
    end_step_id = step_seq[-1]
    start_node_id = step2node[start_step_id]
    end_node_id = step2node[end_step_id]
    return start_node_id, end_node_id

def get_node_seq_of_task(task_name):
    a_id = wikihow_taskname_to_article_id[task_name]
    step_seq = article_to_step_id[a_id]
    node_id_seq = [step2node[step_id] for step_id in step_seq]
    return node_id_seq

# TODO: It makes sense on the local graph
def get_start_node_of_task_via_vid(vid):
    # start step of vid
    # 
    step_seq = vid_to_step_id[vid]
    start_step_id = step_seq[0]
    start_node_id = step2node[start_step_id]
    return start_node_id

In [92]:
import sys
sys. setrecursionlimit(10000) 

In [90]:
# step 2: on selective task names
for t in in_wikihow:
    print('Task title:', t)
    
    print('Domain:', tasktitles2domain.get(t))

    start_node, end_node = get_start_end_node_of_task(t)
    cae_node_trans = find_node_transitions(cae_pkg, start_node=start_node)
    print('Total node transistions found', len(cae_node_trans))
    cae_step_trans = get_step_sequence_by_node_sequence(cae_node_trans, node2step, step_id_to_headline)

    # TODO: rewrite, make cae_step_trans on one
    choie = 2 if len(cae_node_trans) > 2 else len(cae_node_trans) 
    num = 0 
    if len(cae_node_trans) > 2:
        for node_tran, step_tran in zip(cae_node_trans, cae_step_trans):
            num += 1
            output = {node: step_tran[idx] for idx, node in enumerate(node_tran)}
            print(output)
            if num > 2:
                break
    else:
        for node_tran, step_tran in zip(cae_node_trans, cae_step_trans):
            output = {node: step_tran[idx] for idx, node in enumerate(node_tran)}
            print(output)

#     howto100m_trans = find_topk_node_transitions(howto_pkg, start_node=start_node, depth_count=10)
#     print(len(howto100m_trans))


Task title: How to Install a Speedometer
Domain: Cars & Other Vehicles
Total node transistions found 285
{4684: ["Read your vehicle's owner's manual."], 9801: ['Find an appropriate speedometer.'], 154: ['Gather the appropriate tools and materials.', 'Gather the necessary tools.', 'Gather the necessary tools.'], 10: ['Disconnect the battery.', 'Disconnect the battery cables.', 'Disconnect the battery.', 'Disconnect the battery.', 'Disconnect the battery.', 'Disconnect the battery.', 'Disconnect the battery.', 'Disconnect the positive battery cable.', 'Disconnect your vehicle’s battery.', 'Disconnect the battery.', 'Disconnect the battery.', 'Disconnect the battery.', 'Disconnect the battery leads.', "Disconnect the negative power cable from the car's battery.", 'Disconnect the battery ground terminal.', 'Disconnect the battery.', 'Disconnect the negative battery cable.', 'Disconnect the negative terminal of your battery.', 'Disconnect the negative battery cable;', 'Disconnect the batter

In [89]:
node2step[10][0]

array([  530,   906,  1003,  1148,  1788,  1837,  2469,  4438,  5060,
        5529,  6703,  7281,  7870,  8065,  8136,  8226,  8428,  9047,
        9698, 10101, 10126, 10209])

# (Outdated) Analysis on PKG constructed base on cross-modal alignment (6 videos)

### Case A: irrelevant task id 

In [151]:
# get action-step sequences based on task id
# Case A: non-relevant task id
example_task_name = 'How to Plant Parsnips'
example_task_id = wikihow_task2taskid[example_task_name]
example_a_id = wikhow_taskname_to_article_id[example_task_name]

# get step sequence (step id transitions)
step_seq = article_to_step_id[example_a_id]
start_step_id = step_seq[0]
start_node_id = step2node[start_step_id]

node_trans = find_node_transitions(pkg, start_node=start_node_id)
print(node_trans)

get_step_sequence_by_node_sequence(node_trans, node2step, step_id_to_headline)

[[3059, 7793, 7260, 1631, 7054, 3334]]


[[['Prepare the site.'],
  ['Lay a piece of string on the soil as a guide to make sure you dig the row in a straight line.'],
  ['Use a cane, or other tool, to dig a narrow hole that you will place the seeds in.'],
  ['Using a hoe, gently cover the seeds with soil.'],
  ['If the soil is dry, water the seeds in.'],
  ['To protect your plants from late frosts in Spring, build a frame to cover the drills.']]]

### Case B: relevant task id 

In [20]:
# videoid2taskid
howto_meta_dir = '/mount/arbeitsdaten/jp-silberer/ernie_vil/data/HowTo100M'
video_meta_csv = pd.read_csv(os.path.join(howto_meta_dir, 'HowTo100M_v1.csv'))
task_ids_csv = pd.read_csv(os.path.join(howto_meta_dir, 'task_ids.csv'), sep='\t', header=None)
    
def get_videoid2taskname(video_meta_csv, task_ids_csv):
    task_id_to_task_name_original_map = dict()
    for index, row in task_ids_csv.iterrows():
        task_id = row[0]
        task_name = row[1]
        task_id_to_task_name_original_map[task_id] = task_name
    
    video_id_to_task_name = dict()
    for index, row in video_meta_csv.iterrows():
        video_id = row['video_id']
        task_id = row['task_id']
        video_id_to_task_name[video_id] = task_id_to_task_name_original_map[task_id]
    
    return video_id_to_task_name

video_id_to_task_name = get_videoid2taskname(video_meta_csv, task_ids_csv)

In [19]:
# sample videos
sample_videos = ['8JevEqO_iS0', 'L8kRaQ-IwpM', 'O_sC9gyEER0', 'c0VlxPBSMDs', 'jRkJTww8HR0', 'kHb7POoH_gc']
sample_task_names = [video_id_to_task_name.get(vid)for vid in sample_videos]
sample_task_names

['Grill Tri Tip',
 'Grill Tri Tip',
 'Do a Hair Mask for Frizzy Hair',
 'Make an Envelope Advent Calendar',
 'Make Pine Needle Tea',
 'Grill Tri Tip']

In [88]:
similar_tasks = []
for t in tasknames:
    if 'Grill' in t:
        similar_tasks.append(t)
similar_tasks

['How to Grill Tri Tip']

In [26]:
example_task_name = 'How to Grill Tri Tip'
example_task_id = wikihow_task2taskid[example_task_name]
example_a_id = wikihow_taskname_to_article_id[example_task_name]

# get step sequence (step id transitions)
step_seq = article_to_step_id[example_a_id]
start_step_id = step_seq[0]
# start_step_id = 4939
start_node_id = step2node[start_step_id]
# print(start_node_id)
node_trans = find_node_transitions(pkg, start_node=start_node_id)
print('node transition', node_trans)

# Find article name
print('step transition', [node2step[n] for n_seq in node_trans for n in n_seq])


get_step_sequence_by_node_sequence(node_trans, node2step, step_id_to_headline)


node transition [[1956, 1190, 1451, 619, 937, 1896]]
step transition [array([9463]), array([9464]), array([9465]), array([9466, 9473]), array([9474]), array([9475])]


[[['Head to your grocery store.'],
  ['Prep the roast.'],
  ['Use a dry rub.'],
  ['Give it a rest.', 'Give it another rest.'],
  ['Slice against the grain.'],
  ['Serve with your favorite sides.']]]

In [108]:
idx = np.nonzero(wikihow_step2task[4939]!=0)[0]
print(idx)
wikihow_taskid2task[556]

[556]


'How to Smoke Beef Ribs1'

### Q. Why there is no other sequence transitions?

In [ ]:
# Find if the node_trans is in G_Howto100M; if yes, the order align with the wikihow's step headlines
# If not; that means, the alignment between the video segment and the step headlines are not well (Check how S3D is trained)
# Find the similarity score of 8JevEqO_iS0, L8kRaQ-IwpM, kHb7POoH_gc to get some insights


In [8]:
G_howto100m = np.load(os.path.join(cae_dir, 'subtitles/graph_output/G_howto100m-criteria_topK-threshold_10-topK_3-agg_1000.npy')) # on step-level transition
G_wikihow = np.load(os.path.join(cae_dir, 'subtitles/graph_output/G_wikihow-criteria_topK-threshold_10-topK_3.npy')) # on step-level transition

In [65]:
from scipy.sparse import csr_matrix
G_wikihow_csr, G_howto100m_csr = csr_matrix(G_wikihow), csr_matrix(G_howto100m)

In [75]:
G_wikihow_step_trans = np.argwhere(G_wikihow>0)
index = np.nonzero(G_wikihow)
G_wikihow_conf = G_wikihow[index]

formed_edges_in_nodes = [(step2node[formed_edge_in_steps[0]], step2node[formed_edge_in_steps[1]]) for formed_edge_in_steps in G_wikihow_step_trans]

True


In [78]:
print((937, 1896) in formed_edges_in_nodes)

True


In [18]:
G_howto100m_step_trans = np.argwhere(G_howto100m>0)
index = np.nonzero(G_howto100m)
G_howoto100m_conf = G_howto100m[index]


for conf, formed_edge_in_steps in zip(G_howoto100m_conf, G_howto100m_step_trans):
    print('step transition:', formed_edge_in_steps)
    print('step transition in headlines:', step_id_to_headline[formed_edge_in_steps[0]], step_id_to_headline[formed_edge_in_steps[1]])
    print('confidence score:', conf)

step transition: [ 759 7090]
step transition in headlines: Meanwhile, get the fermentors. Inspect the clean roof for cracks along the edges of the EPDM barrier where the self leveling sealant is starting to lift up or become degraded in any way.
confidence score: 0.8663709492554348
step transition: [ 759 7093]
step transition in headlines: Meanwhile, get the fermentors. Apply this sealant along the cracked or potential pin holed area that you have just cleaned from any dirt and grime.
confidence score: 0.8671828954962901
step transition: [ 759 9266]
step transition in headlines: Meanwhile, get the fermentors. Crack all the grains (Meaning should lightly grind up in coffee grinder, or roll over with a heavy rolling pin while in sleeve).
confidence score: 0.8734952834486645
step transition: [1498 3599]
step transition in headlines: Clean the hub, outer bearing and inner bearing with parts cleaner or a brush and pan of kerosene or gasoline. Make a nesting box out of a kitchen cabinet.
con

In [ ]:
# how to find videoid2stepid, having this would be helpful to find a step sequence based on the start of a given video id
# maybe not set threshold on PKG edge transition